# 规划和股票报告生成
在这章我们将会搭建一个定制的多Agent组成的群聊，协作生成关于过去一个月股票表现的详细报告。
<center>
<img src="images/group_of_agents.jpg" width="600">
</center>

## 设置配置

In [1]:
llm_config={"model": "gpt-4-turbo"}

## 创建任务

In [2]:
task = "写一篇关于过去一个月"\
"Nvidia股价表现的博客。今天是 2024-04-23。"

## 建立群聊

在之前的学习中，我们学习了顺序聊天的对话模版，它可以执行多步任务，但需要人类设计具体的步骤和每个步骤中涉及的具体Agent，如果我们想要简单地定义Agent并让它们共同解决这个任务，那么这个群聊的对话模式能够满足你的需求。

为了完成任务，这个群聊将包含以下Agent：

1. **User_proxy** or **Admin**: 允许用户对报告进行评论，并要求作者对报告进行改进。
2. **Planner**: 确定完成任务所需的相关信息。
3. **Engineer**: 使用Planner定义的计划编写代码。
4. **Executor**: 执行Engineer编写的代码。
5. **Writer**: 撰写报告
接下来我们逐个定义上述Agengt。

In [3]:
import autogen

In [4]:
user_proxy = autogen.ConversableAgent(
    name="Admin",                       # 管理员
    system_message="Give the task, and send "# 给出任务，并发送指令给作者，从而完善博客文章。
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",          # 将人类输入模式设置为 ALWAYS 因此每次轮到Admin Agent说话时，始终要求人类输入
)                                       # 如果跳过输入，Admin Agent会发送指示给Writer Agengt来改进博文

[autogen.oai.client: 06-14 21:04:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
planner = autogen.ConversableAgent(
    name="Planner",                                 # 规划者
    system_message="Given a task, please determine " # 给定一项任务，请确定完成该任务需要哪些信息。
    "what information is needed to complete the task. "# 请注意，所有信息都将使用Python代码检索。
    "Please note that the information will all be retrieved using"# 请只建议可以使用Python代码检索的信息。
    " Python code. Please only suggest information that can be "# 在其他人完成每一步后，检查进度并指示剩余步骤。
    "retrieved using Python code. "                              # 如果某一步失败，尝试解决。
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "# 规划者。给定一项任务，确定完成该任务需要哪些信息。
    "information is needed to complete the task. "# 在其他人完成每一步后，检查进度并指示剩余步骤
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

[autogen.oai.client: 06-14 21:04:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
engineer = autogen.AssistantAgent(
    name="Engineer",            # 工程师                   
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "# 基于规划者的计划写代码。
    "provided by the planner.",
)

[autogen.oai.client: 06-14 21:04:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
executor = autogen.ConversableAgent(
    name="Executor",                        # 执行者
    system_message="Execute the code written by the "# 执行工程师写的代码并且返回结果。
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,   # 能看到的执行代码结果的消息数
        "work_dir": "coding",   # 代码保存路径
        "use_docker": False,    # 是否使用docker执行代码
    },
)

In [8]:
writer = autogen.ConversableAgent(
    name="Writer",                      # 作家/写作者
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"  # 请用markdown格式写博客(带有相关标题)，并将内容放在伪``` `md```代码块中。
    " and put the content in pseudo ```md``` code block. "      # 你从管理员那里得到反馈并改进你的博客。
    "You take feedback from the admin and refine your blog.",
    description="Writer."                                   # 作家/写作者
    "Write blogs based on the code execution results and take "     # 根据代码执行结果编写博客，并根据管理员的反馈完善博客
    "feedback from the admin to refine the blog."
)

[autogen.oai.client: 06-14 21:04:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## 定义群聊
将之前定义好的Agent拉进群聊。

In [9]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],   # agents为参与群聊的Agent，接收格式为List[Agent]
    messages=[],   
    max_round=10,   
)

定义群聊管理器，我们要通过Admin对群聊管理器提出任务从而开始对话，同样的群聊管理器也需要配置 `llm_config`。

In [10]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


[autogen.oai.client: 06-14 21:04:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## 开始群聊

In [11]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

写一篇关于过去一个月Nvidia股价表现的博客。今天是 2024-04-23。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

为了完成关于过去一个月NVIDIA股价表现的博客，你需要做以下几个步骤：

1. **获取股票数据**：
   - 你需要从2024年3月23日到2024年4月22日的NVIDIA股票数据。
   - 数据应包括至少每日的开盘价、最高价、最低价和收盘价。

2. **分析股票表现**：
   - 比较3月23日和4月22日的收盘价，评估股价变化。
   - 计算此期间的股价最高值和最低值。
   - 可以计算平均收盘价或其他相关的统计数据。

3. **撰写博客文章**：
   - 根据收集的数据分析NVIDIA过去一个月的股价表现。
   - 提供量化数据支持你的分析，如价格变动、最高点和最低点。
   - 讨论可能影响股价变动的外部因素，如市场新闻或技术创新。

### 第一步：获取股票数据
为了获取股票数据，你可以使用Python中的几种金融数据库接口，如 `yfinance`。可以通过下面的Python代码来获取NVIDIA（股票代码为"NVDA"）的股票数据：

```python
import yfinance as yf
import pandas as pd

# 设置日期范围
start_date = "2024-03-23"
end_date = "2024-04-22"

# 下载NVDA股票数据
nvda_data = yf.download('NVDA', start=start_date, end=end_date)

# 显示数据
print(nvda_data)
```

该代码段将输出NVIDIA股票的每日开盘价、最高价、最低价和收盘价等数据。

#### 接下来的步骤：
- 执行上述Python代码来获取NVIDIA股票数据。
- 检查数据是否已正确获取并呈现所需日期范围内的数据。如果成功，请告诉我以便继续进行下一步分析。如果

## 添加说话人选择策略

In [12]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "   # 给出任务，并发送指令给作者，从而完善博客文章。
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "     # 给定一项任务，请确定完成该任务需要哪些信息。
    "what information is needed to complete the task. " # 请注意，所有信息都将使用Python代码检索。
    "Please note that the information will all be retrieved using"  # 请只建议可以使用Python代码检索的信息。
    " Python code. Please only suggest information that can be "    # 在其他人完成每一步后，检查进度并指示剩余步骤。
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "     # 如果某一步失败，尝试解决。
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "     # 规划者。给定一项任务，确定完成该任务需要哪些信息。
    "information is needed to complete the task. "  # 在其他人完成每一步后，检查进度并指示剩余步骤
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "     # 基于规划者的计划写代码。
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"  # 请用markdown格式写博客(带有相关标题)，并将内容放在伪``` `md```代码块中。
    " and put the content in pseudo ```md``` code block. "      
    "You take feedback from the admin and refine your blog.",       # 你从管理员那里得到反馈并改进你的博客。
    description="After all the info is available, "
    "write blogs based on the code execution results and take "     # 根据代码执行结果编写博客，并根据管理员的反馈完善博客
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "      # 执行工程师编写的代码并报告结果。
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

[autogen.oai.client: 06-14 21:05:55] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 06-14 21:05:55] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 06-14 21:05:56] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 06-14 21:05:56] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


我们上次定义群聊可能会出现发言混乱的情况，我们可以通过添加`allowed_or_disallowed_speaker_transitions`与`speaker_transitions_type`参数来对群聊进行额外的约束，前者可以指定Agent的消息能够/不能够被哪些Agent看到，具体是能够或不能够则可以通过设置`speaker_transitions_type`为`allowed`或`disallowed`来控制。

In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [14]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

[autogen.oai.client: 06-14 21:05:56] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Admin (to chat_manager):

写一篇关于过去一个月Nvidia股价表现的博客。今天是 2024-04-23。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

为了完成关于过去一个月NVIDIA股价表现的博客，你需要做以下几个步骤：

1. **获取股票数据**：
   - 你需要从2024年3月23日到2024年4月22日的NVIDIA股票数据。
   - 数据应包括至少每日的开盘价、最高价、最低价和收盘价。

2. **分析股票表现**：
   - 比较3月23日和4月22日的收盘价，评估股价变化。
   - 计算此期间的股价最高值和最低值。
   - 可以计算平均收盘价或其他相关的统计数据。

3. **撰写博客文章**：
   - 根据收集的数据分析NVIDIA过去一个月的股价表现。
   - 提供量化数据支持你的分析，如价格变动、最高点和最低点。
   - 讨论可能影响股价变动的外部因素，如市场新闻或技术创新。

### 第一步：获取股票数据
为了获取股票数据，你可以使用Python中的几种金融数据库接口，如 `yfinance`。可以通过下面的Python代码来获取NVIDIA（股票代码为"NVDA"）的股票数据：

```python
import yfinance as yf
import pandas as pd

# 设置日期范围
start_date = "2024-03-23"
end_date = "2024-04-22"

# 下载NVDA股票数据
nvda_data = yf.download('NVDA', start=start_date, end=end_date)

# 显示数据
prin

**注意**: 你可能会体验到智能体之间略有不同的交互。工程师Agent可能编写了不正确的代码，执行Agent将报告并发回更正。这个过程可能会经过多轮。